In [ ]:
import pickle
import pandas as pd
import numpy as np

In [ ]:
import os
def get_testing_data(directory,min_length,window_size,maxlen = float('inf')):
    X = []
    y = []
    n = 60//window_size
    count  = 0
    for f in os.listdir(directory):
        if f[0]=='.':
            continue
        data = pickle.load(open(directory+f,'rb'))
        if data.shape[0]//n<min_length:
            continue
        count+=data.shape[0]
        
        X.append(np.concatenate(list(data['data'])))
        y.extend([f]*data.shape[0])
        if count>=maxlen:
            break
    return np.concatenate(X),np.array(y)
activities = ['stationery','stairs','sports','walking']
min_length = 100
window_size = 20
maxlen = 10000000
std_df = []
for activity in activities:
    data_directory = './data/'+str(window_size)+'/'+activity+'/testing/'
    X,y = get_testing_data(data_directory,min_length,window_size,maxlen=maxlen)
    X_magnitude = np.array([np.sqrt(np.sum(np.square(a),axis=1)) for a in X])
    X_std = np.std(X_magnitude,axis=1)
#     if activity in ['stairs','walking']:
#         activity = 'Walking & Stairs'
    if activity in ['stairs','walking','exercise']:
        X_std = X_std[X_std>=.025]
#     elif activity == 'stationery':
#         X_std = X_std[X_std>=.005]
    std_df.append(pd.DataFrame({'std':np.array(X_std),'activity':[activity]*len(X_std)}))
df = pd.concat(std_df)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':30})
plt.figure(figsize=(15,8))
sns.boxplot(x='activity',y='std',data=df,showfliers=False)
plt.xlabel('Activity Type')
plt.ylabel('Accelerometer\n Standard Deviation')
plt.tight_layout()
plt.savefig('./images/boxplot_of_std_activities.png')
plt.show()

In [ ]:
df['Activity Type'] = df['activity']
plt.rcParams.update({'font.size':30})
plt.figure(figsize=(15,8))
sns.histplot(df,x='std',hue='Activity Type',stat='density',kde=True)
plt.xlim([0,.5])
plt.ylim([0,15])
plt.xlabel('Accelerometer\n Standard Deviation')
plt.tight_layout()
plt.savefig('./images/histogram_of_std_activities.png')
plt.show()